# QML Challenge

In questa sfida, i partecipanti svilupperanno un modello per prevedere i livelli di PM25 sulla base delle misurazioni di umidità e PM10. Questa sfida simula uno scenario reale in cui si vuole stimare il particolato fine (PM25) utilizzando parametri più facilmente misurabili.

In [ ]:
%pip install seaborn pandas scikit-learn torch pennylane qbraid
# Use the CRS4 environment for this challenge!

import os
import pathlib

import pandas as pd
import torch
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns
# read in the data set
from client_grader import * 
grader=Crs4GraderClient()
df = pd.read_csv('./particolato.csv',index_col=0, dtype={ "sensors.humidify": "int64", "sensors.pm10":"float64", "sensors.pm25":"float64"}, parse_dates=["Time"])




In [ ]:
# okay lets try doing some EDA on this 

df.dropna(inplace=True)
# Display the first few rows
print(df.head())

# Basic information about the dataset
print(df.info())

# Summary statistics
print(df.describe())

In [ ]:
# EDA
plt.figure(figsize=(12, 8))
sns.pairplot(df[['sensors.humidity', 'sensors.pm10', 'sensors.pm25']])
plt.tight_layout()
plt.show()

# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df[['sensors.humidity', 'sensors.pm10', 'sensors.pm25']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()


In [ ]:
# Time series plot
plt.figure(figsize=(12, 6))
plt.plot( df['sensors.pm25'], label='PM2.5')
plt.plot( df['sensors.pm10'], label='PM10')
plt.title('PM2.5 and PM10 over Time')
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.legend()
plt.show()


In [5]:

# Feature Engineering
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek

In [6]:
# Split the data
X = df[['sensors.humidity', 'sensors.pm10', 'hour', 'day_of_week']]
y = df['sensors.pm25']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)


In [ ]:
# Set default tensor type to float32
torch.set_default_tensor_type(torch.FloatTensor)

# Define the quantum device
n_features = X_train.shape[1]
print(f"Number of features: {n_features}")
print(f"Number of qubits: {min(n_features, 10)}")

n_qubits = min(n_features, 10)  # Limit the number of qubits to a maximum of 10

# Define the quantum circuit
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def quantum_circuit(inputs, weights):
    # YOUR CODE HERE
    # --------------
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    # --------------
    # Return the expectation value of Pauli-Z
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

class HybridModel(torch.nn.Module):
    def __init__(self, n_features, n_qubits):
        super().__init__()
        self.n_features = n_features
        self.n_qubits = n_qubits
        self.pre_net = torch.nn.Linear(n_features, n_qubits)
        self.q_params = torch.nn.Parameter(torch.randn(3, n_qubits))
        self.post_net = torch.nn.Linear(n_qubits, 1)


    def forward(self, x):
        x = self.pre_net(x)
        x = torch.stack([torch.tensor(quantum_circuit(x_i.unsqueeze(0) if x_i.dim() == 0 else x_i, self.q_params), dtype=torch.float32) for x_i in x])
        x = self.post_net(x)
        return x

# Initialize the model
model = HybridModel(n_features, n_qubits)

In [ ]:
# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Ensure input data is float32
X_train_tensor = X_train_tensor.float()
y_train_tensor = y_train_tensor.float()
X_test_tensor = X_test_tensor.float()
y_test_tensor = y_test_tensor.float()

# Training loop
epochs = 10
batch_size = 32

for epoch in range(epochs):
    model.train()
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_test_tensor)
            val_loss = criterion(val_outputs, y_test_tensor)
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)

mae = mean_absolute_error(y_test, y_pred.numpy())
print(f"Mean Absolute Error: {mae:.4f}")

In [ ]:

# Plot actual vs predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual PM2.5')
plt.ylabel('Predicted PM2.5')
plt.title('Actual vs Predicted PM2.5')
plt.show()

In [10]:

torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()  

In [12]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=.7, random_state=42)

# Scale the validation and test sets
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.FloatTensor(y_val.values).reshape(-1, 1)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)

In [13]:
def compute_accuracy(y_true, y_pred, threshold=0.2):
    correct = torch.abs(y_true - y_pred) < threshold
    print(correct)
    return correct.float().mean().item()

In [ ]:
model.eval()
with torch.no_grad():
    y_val_pred = model(X_val_tensor)
    val_accuracy = compute_accuracy(y_val_tensor, y_val_pred)
    
print(f"Validation Accuracy (threshold=0.2): {val_accuracy:.4f}")

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

val_mae = mean_absolute_error(y_val, y_val_pred.numpy())
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred.numpy()))

print(f"Validation MAE: {val_mae:.4f}")
print(f"Validation RMSE: {val_rmse:.4f}")

In [ ]:
with torch.no_grad():
    y_test_pred = model(X_test_tensor)
    test_accuracy = compute_accuracy(y_test_tensor, y_test_pred)
    
test_mae = mean_absolute_error(y_test, y_test_pred.numpy())
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred.numpy()))

print(f"Test Accuracy (threshold=0.2): {test_accuracy:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_test_pred.numpy(), alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual PM2.5')
plt.ylabel('Predicted PM2.5')
plt.title('Actual vs Predicted PM2.5 (Test Set)')
plt.show()

## Sottomissione della sfida per ottenere il punteggio

Abbiamo integrato il sistema di punteggio nella piattaforma qBraid.

In [ ]:
# Get the path to the model weights
model_path = 'model_weights.pth'
# Check if the model weights file exists
if os.path.exists(model_path):
    # Load the model
    model.load_state_dict(torch.load(model_path))
    print("Model weights loaded successfully!")

# Get the full path to the model weights file
model_path = pathlib.Path(model_path).resolve()
print(str(model_path))

grader.check_submission(str(model_path), '13.1')
result = grader.submit_exercise(str(model_path), '13.1')
print(result)

L'elaborazione dei risultati potrebbe richiedere del tempo.


Dopo qualche minuto, controllate [la classifica in tempo reale](https://account.qbraid.com/hackathons/2024/crs4) per vedere come si classifica il vostro team!

🍀 In bocca al lupo! Speriamo che tu ti diverta a cimentarti in questa sfida .🤞 🥳 🎉


# 👉 [La classifica in tempo reale](https://account.qbraid.com/hackathons/2024/crs4)

Visitate il link qua sopra per vedere che punteggio avete ottenuto.